In [29]:
import psycopg2
import json
import ipywidgets as widgets
from ipyleaflet import Map, Marker, Circle, GeoJSON, basemaps, WidgetControl
from IPython.display import display
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

# Database connection parameters - UPDATE THESE
DB_CONFIG = {
    'host': '192.168.1.41',
    'database': 'dbsm',
    'user': 'postgres',
    'password': 'antarone',
    'port': 5432
}

# Your PostGIS function name - UPDATE THIS
POSTGIS_FUNCTION = "public.__get_buildings_in_buffer_geojson"  # e.g., "get_nearby_features"

In [30]:
class PostGISInteractiveMap:
    def __init__(self):
        self.markers = []
        self.circles = []
        self.geojson_layers = []
        self.setup_widgets()
        self.setup_map()
        self.setup_interactions()
        
    def setup_widgets(self):
        """Create interactive widgets"""
        self.buffer_slider = widgets.FloatSlider(
            value=1000.0,
            min=100.0,
            max=5000.0,
            step=50.0,
            description='Buffer (m):',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.auto_query_checkbox = widgets.Checkbox(
            value=True,
            description='Auto-query on click',
            style={'description_width': 'initial'}
        )
        
        self.query_button = widgets.Button(
            description='Query PostGIS',
            button_style='info',
            icon='search',
            layout=widgets.Layout(width='150px')
        )
        
        self.clear_button = widgets.Button(
            description='Clear Results',
            button_style='warning',
            icon='eraser',
            layout=widgets.Layout(width='150px')
        )
        
        self.status_output = widgets.Output(
            layout=widgets.Layout(height='60px', overflow='auto')
        )
        
        self.coordinate_display = widgets.HTML(
            value="<b>Click on map to select coordinates</b>",
            layout=widgets.Layout(width='300px')
        )
        
        # Connect events
        self.query_button.on_click(self.on_query_button_click)
        self.clear_button.on_click(self.on_clear_button_click)
        self.buffer_slider.observe(self.on_buffer_change, names='value')
        
        # Current coordinates
        self.current_lat = None
        self.current_lon = None
        
    def setup_map(self):
        """Initialize the ipyleaflet map"""
        self.map = Map(
            basemap=basemaps.OpenStreetMap.Mapnik,
            center=[41.9028, 12.4964],  # Rome, Italy
            zoom=10,
            layout=widgets.Layout(height='600px')
        )
        
        # Add control panel to map
        control_panel = widgets.VBox([
            widgets.HTML("<h4>PostGIS Query Tool</h4>"),
            self.coordinate_display,
            self.buffer_slider,
            self.auto_query_checkbox,
            widgets.HBox([self.query_button, self.clear_button]),
            self.status_output
        ])
        
        # Add control panel to map
        widget_control = WidgetControl(widget=control_panel, position='topright')
        self.map.add_control(widget_control)
        
    def setup_interactions(self):
        """Setup map interaction handlers"""
        self.map.on_interaction(self.handle_map_click)
        
    def handle_map_click(self, **kwargs):
        """Handle map click events"""
        if kwargs.get('type') == 'click':
            coordinates = kwargs.get('coordinates')
            self.current_lat, self.current_lon = coordinates
            
            # Update coordinate display
            self.coordinate_display.value = f"<b>Selected:</b> {self.current_lat:.4f}, {self.current_lon:.4f}"
            
            # Auto-query if enabled
            if self.auto_query_checkbox.value:
                self.query_postgis_data()
                
    def connect_to_postgis(self):
        """Create database connection"""
        try:
            conn = psycopg2.connect(**DB_CONFIG)
            return conn
        except Exception as e:
            with self.status_output:
                print(f"❌ Database connection error: {str(e)}")
            return None
            
    def query_postgis_function(self, lat, lon, buffer_distance):
        """Query the PostGIS function"""
        conn = self.connect_to_postgis()
        if not conn:
            return None
            
        try:
            cursor = conn.cursor()
            
            # Call your PostGIS function
            query = f"""
                SELECT {POSTGIS_FUNCTION}(%s, %s, %s) as geojson;
            """
            
            cursor.execute(query, (lat, lon, buffer_distance))
            result = cursor.fetchone()

            print(result)
            
            if result and result[0]:
                return result[0]
            else:
                return None
                
        except Exception as e:
            with self.status_output:
                print(f"❌ Query error: {str(e)}")
            return None
        finally:
            if conn:
                conn.close()
                
    def clear_results(self):
        """Clear all results from map"""
        # Remove markers
        for marker in self.markers:
            self.map.remove_layer(marker)
        self.markers.clear()
        
        # Remove circles
        for circle in self.circles:
            self.map.remove_layer(circle)
        self.circles.clear()
        
        # Remove GeoJSON layers
        for layer in self.geojson_layers:
            self.map.remove_layer(layer)
        self.geojson_layers.clear()
        
    def add_results_to_map(self, geojson_data, lat, lon, buffer_distance):
        """Add query results to map"""
        try:
            # Add query point marker
            marker = Marker(
                location=[lat, lon],
                draggable=False,
                title=f"Query Point - Buffer: {buffer_distance}m"
            )
            self.map.add_layer(marker)
            self.markers.append(marker)
            
            # Add buffer circle
            circle = Circle(
                center=[lat, lon],
                radius=int(buffer_distance),
                color='red',
                fillColor='red',
                fillOpacity=0.2,
                weight=2
            )
            self.map.add_layer(circle)
            self.circles.append(circle)
            
            # Add GeoJSON data
            if isinstance(geojson_data, str):
                geojson_data = json.loads(geojson_data)
                
            geojson_layer = GeoJSON(
                data=geojson_data,
                style={
                    'color': 'blue',
                    'weight': 2,
                    'fillColor': 'blue',
                    'fillOpacity': 0.6
                },
                hover_style={
                    'color': 'green',
                    'weight': 3,
                    'fillOpacity': 0.8
                }
            )
            
            self.map.add_layer(geojson_layer)
            self.geojson_layers.append(geojson_layer)
            
            # Center map on query point
            self.map.center = [lat, lon]
            
            with self.status_output:
                print(f"✅ Query completed for ({lat:.4f}, {lon:.4f}) with {buffer_distance}m buffer")
                
        except Exception as e:
            with self.status_output:
                print(f"❌ Error adding results to map: {str(e)}")
                
    def query_postgis_data(self):
        """Execute PostGIS query with current parameters"""
        if self.current_lat is None or self.current_lon is None:
            with self.status_output:
                print("❌ Please click on the map to select coordinates first")
            return
            
        buffer_distance = self.buffer_slider.value
        
        with self.status_output:
            print(f"🔍 Querying PostGIS function for coordinates ({self.current_lat:.4f}, {self.current_lon:.4f})...")
            
        # Query PostGIS function
        geojson_result = self.query_postgis_function(
            self.current_lat, 
            self.current_lon, 
            buffer_distance
        )
        
        if geojson_result:
            # Clear previous results
            self.clear_results()
            
            # Add new results
            self.add_results_to_map(
                geojson_result, 
                self.current_lat, 
                self.current_lon, 
                buffer_distance
            )
        else:
            with self.status_output:
                print("❌ No results found or query failed")
                
    def on_query_button_click(self, button):
        """Handle manual query button click"""
        self.query_postgis_data()
        
    def on_clear_button_click(self, button):
        """Handle clear button click"""
        self.clear_results()
        self.current_lat = None
        self.current_lon = None
        self.coordinate_display.value = "<b>Click on map to select coordinates</b>"
        
        with self.status_output:
            print("🧹 Map cleared")
            
    def on_buffer_change(self, change):
        """Handle buffer slider change"""
        if self.auto_query_checkbox.value and self.current_lat is not None:
            self.query_postgis_data()
            
    def display(self):
        """Display the map application"""
        instructions = widgets.HTML("""
        <div style="padding: 10px; background-color: #f0f0f0; border-radius: 5px; margin-bottom: 10px;">
            <h3>🗺️ PostGIS Interactive Query Tool</h3>
            <p><strong>Instructions:</strong></p>
            <ul>
                <li>Click anywhere on the map to select query coordinates</li>
                <li>Adjust the buffer distance using the slider</li>
                <li>Toggle auto-query to run queries automatically when clicking or changing buffer</li>
                <li>Use the manual query button if auto-query is disabled</li>
                <li>Clear results to remove all markers and data from the map</li>
            </ul>
        </div>
        """)
        
        display(instructions)
        display(self.map)

In [31]:
# Create and display the application
postgis_app = PostGISInteractiveMap()
postgis_app.display()

HTML(value='\n        <div style="padding: 10px; background-color: #f0f0f0; border-radius: 5px; margin-bottom:…

Map(center=[41.9028, 12.4964], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…